In [35]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)

#train
df = pd.read_csv("./input/train.csv")
df.drop(['start_square', 'plan_s', 'plan_m', 'plan_l', 'vid_0', 'vid_1', 'vid_2'], axis=1, inplace=True)
df.to_csv("./input/train_full.csv", index=False)

#test
dt = pd.read_csv("./input/test.csv")
dt['value'] = 0
dt.to_csv("./input/test_.csv", index=False, columns = df.columns)

In [36]:
#valid
from sklearn.model_selection import KFold
import pandas as pd
df = pd.read_csv("./input/train_full.csv")

kf = KFold(n_splits=10, random_state=42, shuffle=True)
kf.get_n_splits(range(len(df)))

print(kf)  

for i, (train_index, valid_index) in enumerate(kf.split(range(len(df)))):
    train = df.loc[train_index]
    valid = df.loc[valid_index]

    train.to_csv(f"./input/train_{i}.csv",index=False)
    valid.to_csv(f"./input/valid_{i}.csv",index=False)

KFold(n_splits=10, random_state=42, shuffle=True)


In [ ]:
K = 10

from catboost import CatBoostRegressor, Pool
import pandas as pd
import numpy as np

path = './input/'

TEST_FILE = path + 'test_.csv' #add
CD_FILE = path + 'train.cd'

test_pool = Pool(TEST_FILE, column_description=CD_FILE, has_header=True, delimiter=",")

df = pd.read_csv('./input/train_full.csv')
oof = np.zeros((len(df), 1))

dt = pd.read_csv(TEST_FILE)
test = np.zeros((len(dt), 1))

for i in range(K):

    TRAIN_FILE = path+f'train_{i}.csv'
    VAL_FILE = path+f'valid_{i}.csv'

    train_pool = Pool(TRAIN_FILE, column_description=CD_FILE, has_header=True, delimiter=",")
    val_pool = Pool(VAL_FILE, column_description=CD_FILE, has_header=True, delimiter=",")

    model = CatBoostRegressor(iterations=5000, 
                              learning_rate=0.05, 
                              depth=10, 
                              random_seed = 42)
    
    model.fit(train_pool, eval_set = val_pool)
    
    pred = model.predict(test_pool)
    test += pred.reshape((len(dt),1))
    dt['value'] = pred
    dt.to_csv(f"./output/test_{i}.csv", index=False, columns=['id','value'])
    
    pred = model.predict(val_pool)
    df = pd.read_csv(VAL_FILE)
    oof[df.id] = pred.reshape((len(df),1))
    df['value'] = pred
    df.to_csv(f"./output/oof_{i}.csv", index=False, columns=['id','value'])
    
df = pd.read_csv(path+"sample submission.csv")
df['value'] = (test/K).clip(0, 5000)
df.to_csv("./output/sub.csv", index=False)

df = pd.read_csv(path+"train_full.csv")
df['value_pred'] = oof
df.to_csv("./output/oof.csv", index=False)

0:	learn: 453.0494799	test: 444.4819109	best: 444.4819109 (0)	total: 359ms	remaining: 29m 55s
1:	learn: 444.1769824	test: 435.4429211	best: 435.4429211 (1)	total: 387ms	remaining: 16m 6s
2:	learn: 434.7659387	test: 425.7938955	best: 425.7938955 (2)	total: 748ms	remaining: 20m 45s
3:	learn: 425.6154983	test: 417.0699106	best: 417.0699106 (3)	total: 1s	remaining: 20m 49s
4:	learn: 416.7858117	test: 408.4843643	best: 408.4843643 (4)	total: 1.22s	remaining: 20m 23s
5:	learn: 408.7718990	test: 400.4890415	best: 400.4890415 (5)	total: 1.48s	remaining: 20m 34s
6:	learn: 401.0619935	test: 393.1011432	best: 393.1011432 (6)	total: 1.68s	remaining: 19m 59s
7:	learn: 394.4036844	test: 386.2239364	best: 386.2239364 (7)	total: 1.92s	remaining: 19m 59s
8:	learn: 388.1115805	test: 379.9646083	best: 379.9646083 (8)	total: 2.18s	remaining: 20m 9s
9:	learn: 382.3670473	test: 374.1488542	best: 374.1488542 (9)	total: 2.23s	remaining: 18m 34s
10:	learn: 377.7015555	test: 369.5653472	best: 369.5653472 (10)	t

87:	learn: 261.8152108	test: 263.5008919	best: 263.5008919 (87)	total: 17.8s	remaining: 16m 34s
88:	learn: 261.2160663	test: 263.3616646	best: 263.3616646 (88)	total: 17.9s	remaining: 16m 26s
89:	learn: 260.5062712	test: 262.6125008	best: 262.6125008 (89)	total: 18.1s	remaining: 16m 29s
90:	learn: 259.8674953	test: 262.0070381	best: 262.0070381 (90)	total: 18.3s	remaining: 16m 28s
91:	learn: 259.6965356	test: 261.9357116	best: 261.9357116 (91)	total: 18.6s	remaining: 16m 30s
92:	learn: 258.9729935	test: 261.4095604	best: 261.4095604 (92)	total: 18.9s	remaining: 16m 34s
93:	learn: 258.3871023	test: 260.9735910	best: 260.9735910 (93)	total: 19.1s	remaining: 16m 37s
94:	learn: 257.7728646	test: 260.9252436	best: 260.9252436 (94)	total: 19.4s	remaining: 16m 39s
95:	learn: 257.4396885	test: 260.6896176	best: 260.6896176 (95)	total: 19.4s	remaining: 16m 32s
96:	learn: 256.6610178	test: 260.0251769	best: 260.0251769 (96)	total: 19.7s	remaining: 16m 34s
97:	learn: 256.1762697	test: 259.5897699

172:	learn: 226.7818656	test: 243.6114185	best: 243.6114185 (172)	total: 35.8s	remaining: 16m 37s
173:	learn: 226.3461311	test: 243.2911725	best: 243.2911725 (173)	total: 36s	remaining: 16m 38s
174:	learn: 225.9344117	test: 242.9376717	best: 242.9376717 (174)	total: 36.2s	remaining: 16m 39s
175:	learn: 225.7540859	test: 242.8427563	best: 242.8427563 (175)	total: 36.5s	remaining: 16m 39s
176:	learn: 225.4227457	test: 242.6686502	best: 242.6686502 (176)	total: 36.7s	remaining: 16m 39s
177:	learn: 225.1013365	test: 242.6142857	best: 242.6142857 (177)	total: 36.9s	remaining: 16m 39s
178:	learn: 224.8269951	test: 242.6219668	best: 242.6142857 (177)	total: 37.1s	remaining: 16m 39s
179:	learn: 224.4909581	test: 242.6399791	best: 242.6142857 (177)	total: 37.3s	remaining: 16m 39s
180:	learn: 224.2150538	test: 242.3626800	best: 242.3626800 (180)	total: 37.6s	remaining: 16m 40s
181:	learn: 223.8112084	test: 242.1753230	best: 242.1753230 (181)	total: 37.8s	remaining: 16m 41s
182:	learn: 223.570771

256:	learn: 202.2993020	test: 231.4345948	best: 231.4345948 (256)	total: 57s	remaining: 17m 32s
257:	learn: 201.9027481	test: 231.2932221	best: 231.2932221 (257)	total: 57.2s	remaining: 17m 31s
258:	learn: 201.7897987	test: 231.1251661	best: 231.1251661 (258)	total: 57.4s	remaining: 17m 31s
259:	learn: 201.6889611	test: 231.1198985	best: 231.1198985 (259)	total: 57.8s	remaining: 17m 33s
260:	learn: 201.6141578	test: 231.1037599	best: 231.1037599 (260)	total: 58.1s	remaining: 17m 35s
261:	learn: 201.4835377	test: 230.9759380	best: 230.9759380 (261)	total: 58.4s	remaining: 17m 36s
262:	learn: 201.2525966	test: 230.9903528	best: 230.9759380 (261)	total: 58.7s	remaining: 17m 36s
263:	learn: 200.9599598	test: 230.8344760	best: 230.8344760 (263)	total: 59s	remaining: 17m 38s
264:	learn: 200.5904182	test: 230.7251993	best: 230.7251993 (264)	total: 59.3s	remaining: 17m 38s
265:	learn: 200.3524125	test: 230.6812662	best: 230.6812662 (265)	total: 59.6s	remaining: 17m 41s
266:	learn: 199.9575566	

342:	learn: 187.4079956	test: 224.9147163	best: 224.9147163 (342)	total: 1m 19s	remaining: 18m 5s
343:	learn: 187.3083830	test: 224.8869582	best: 224.8869582 (343)	total: 1m 20s	remaining: 18m 5s
344:	learn: 187.1814994	test: 224.7996069	best: 224.7996069 (344)	total: 1m 20s	remaining: 18m 5s
345:	learn: 187.0587169	test: 224.8450436	best: 224.7996069 (344)	total: 1m 20s	remaining: 18m 4s
346:	learn: 187.0460160	test: 224.8459214	best: 224.7996069 (344)	total: 1m 20s	remaining: 18m 5s
347:	learn: 187.0072862	test: 224.8418298	best: 224.7996069 (344)	total: 1m 21s	remaining: 18m 5s
348:	learn: 186.9285262	test: 224.8647085	best: 224.7996069 (344)	total: 1m 21s	remaining: 18m 4s
349:	learn: 186.6563793	test: 224.5330488	best: 224.5330488 (349)	total: 1m 21s	remaining: 18m 3s
350:	learn: 186.6446435	test: 224.5108375	best: 224.5108375 (350)	total: 1m 21s	remaining: 18m 4s
351:	learn: 186.4561800	test: 224.4351148	best: 224.4351148 (351)	total: 1m 22s	remaining: 18m 3s
352:	learn: 186.2842

...

3246:	learn: 81.7762233	test: 203.8950735	best: 203.8722759 (3243)	total: 13m 52s	remaining: 7m 29s
3247:	learn: 81.7601759	test: 203.8813750	best: 203.8722759 (3243)	total: 13m 52s	remaining: 7m 29s
3248:	learn: 81.7317984	test: 203.8883491	best: 203.8722759 (3243)	total: 13m 53s	remaining: 7m 28s
3249:	learn: 81.7237925	test: 203.8901925	best: 203.8722759 (3243)	total: 13m 53s	remaining: 7m 28s
3250:	learn: 81.7036277	test: 203.8805988	best: 203.8722759 (3243)	total: 13m 53s	remaining: 7m 28s
3251:	learn: 81.6949770	test: 203.8806169	best: 203.8722759 (3243)	total: 13m 53s	remaining: 7m 28s
3252:	learn: 81.6929668	test: 203.8701571	best: 203.8701571 (3252)	total: 13m 53s	remaining: 7m 27s
3253:	learn: 81.6671431	test: 203.8690397	best: 203.8690397 (3253)	total: 13m 54s	remaining: 7m 27s
3254:	learn: 81.6629900	test: 203.8682586	best: 203.8682586 (3254)	total: 13m 54s	remaining: 7m 27s
3255:	learn: 81.5992431	test: 203.8579378	best: 203.8579378 (3255)	total: 13m 54s	remaining: 7m 27s


3329:	learn: 80.2452851	test: 203.6551847	best: 203.6551847 (3329)	total: 14m 15s	remaining: 7m 8s
3330:	learn: 80.2422750	test: 203.6567348	best: 203.6551847 (3329)	total: 14m 15s	remaining: 7m 8s
3331:	learn: 80.2375920	test: 203.6556796	best: 203.6551847 (3329)	total: 14m 15s	remaining: 7m 8s
3332:	learn: 80.2266697	test: 203.6494646	best: 203.6494646 (3332)	total: 14m 16s	remaining: 7m 8s
3333:	learn: 80.2187019	test: 203.6455701	best: 203.6455701 (3333)	total: 14m 16s	remaining: 7m 7s
3334:	learn: 80.2182064	test: 203.6457044	best: 203.6455701 (3333)	total: 14m 16s	remaining: 7m 7s
3335:	learn: 80.2004431	test: 203.6416108	best: 203.6416108 (3335)	total: 14m 16s	remaining: 7m 7s
3336:	learn: 80.1616953	test: 203.6545118	best: 203.6416108 (3335)	total: 14m 17s	remaining: 7m 7s
3337:	learn: 80.1174482	test: 203.6629878	best: 203.6416108 (3335)	total: 14m 17s	remaining: 7m 6s
3338:	learn: 80.1172154	test: 203.6629886	best: 203.6416108 (3335)	total: 14m 17s	remaining: 7m 6s
3339:	lear

3412:	learn: 78.5712111	test: 203.6462616	best: 203.5959917 (3403)	total: 14m 34s	remaining: 6m 46s
3413:	learn: 78.5706333	test: 203.6470757	best: 203.5959917 (3403)	total: 14m 35s	remaining: 6m 46s
3414:	learn: 78.5654501	test: 203.6419560	best: 203.5959917 (3403)	total: 14m 35s	remaining: 6m 46s
3415:	learn: 78.5560430	test: 203.6394119	best: 203.5959917 (3403)	total: 14m 35s	remaining: 6m 46s
3416:	learn: 78.5534795	test: 203.6404049	best: 203.5959917 (3403)	total: 14m 36s	remaining: 6m 45s
3417:	learn: 78.5418132	test: 203.6464518	best: 203.5959917 (3403)	total: 14m 36s	remaining: 6m 45s
3418:	learn: 78.5346424	test: 203.6533059	best: 203.5959917 (3403)	total: 14m 36s	remaining: 6m 45s
3419:	learn: 78.5051990	test: 203.6450961	best: 203.5959917 (3403)	total: 14m 36s	remaining: 6m 45s
3420:	learn: 78.4701340	test: 203.6291663	best: 203.5959917 (3403)	total: 14m 37s	remaining: 6m 44s
3421:	learn: 78.4353711	test: 203.6297743	best: 203.5959917 (3403)	total: 14m 37s	remaining: 6m 44s


3494:	learn: 77.2693844	test: 203.3470476	best: 203.3470476 (3494)	total: 14m 54s	remaining: 6m 25s
3495:	learn: 77.2660593	test: 203.3455335	best: 203.3455335 (3495)	total: 14m 54s	remaining: 6m 24s
3496:	learn: 77.2592747	test: 203.3414773	best: 203.3414773 (3496)	total: 14m 54s	remaining: 6m 24s
3497:	learn: 77.2376112	test: 203.3621450	best: 203.3414773 (3496)	total: 14m 54s	remaining: 6m 24s
3498:	learn: 77.2327743	test: 203.3595959	best: 203.3414773 (3496)	total: 14m 55s	remaining: 6m 23s
3499:	learn: 77.2017104	test: 203.3631243	best: 203.3414773 (3496)	total: 14m 55s	remaining: 6m 23s
3500:	learn: 77.1737630	test: 203.3706894	best: 203.3414773 (3496)	total: 14m 55s	remaining: 6m 23s
3501:	learn: 77.1481934	test: 203.3770964	best: 203.3414773 (3496)	total: 14m 55s	remaining: 6m 23s
3502:	learn: 77.1475947	test: 203.3753495	best: 203.3414773 (3496)	total: 14m 56s	remaining: 6m 22s
3503:	learn: 77.1383123	test: 203.3769557	best: 203.3414773 (3496)	total: 14m 56s	remaining: 6m 22s


3577:	learn: 76.0024160	test: 203.3670844	best: 203.3346956 (3533)	total: 15m 14s	remaining: 6m 3s
3578:	learn: 76.0022125	test: 203.3664616	best: 203.3346956 (3533)	total: 15m 14s	remaining: 6m 3s
3579:	learn: 75.9813429	test: 203.3415573	best: 203.3346956 (3533)	total: 15m 15s	remaining: 6m 3s
3580:	learn: 75.9701115	test: 203.3469700	best: 203.3346956 (3533)	total: 15m 15s	remaining: 6m 2s
3581:	learn: 75.9692482	test: 203.3440007	best: 203.3346956 (3533)	total: 15m 15s	remaining: 6m 2s
3582:	learn: 75.9672995	test: 203.3485473	best: 203.3346956 (3533)	total: 15m 15s	remaining: 6m 2s
3583:	learn: 75.9665240	test: 203.3471120	best: 203.3346956 (3533)	total: 15m 16s	remaining: 6m 1s
3584:	learn: 75.9539277	test: 203.3505722	best: 203.3346956 (3533)	total: 15m 16s	remaining: 6m 1s
3585:	learn: 75.9008168	test: 203.3396878	best: 203.3346956 (3533)	total: 15m 16s	remaining: 6m 1s
3586:	learn: 75.8783173	test: 203.3475782	best: 203.3346956 (3533)	total: 15m 16s	remaining: 6m 1s
3587:	lear

In [ ]:
#добавить грид сёрч